In [1]:
#used anaconda 3.12.3
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image

from model import SimpleCNN, train, test
from customCIFAR10 import CustomCIFAR10
from confMatrix import plot_confusion_matrix

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

LOADING DATASET

    First prepare our data by importing the CIFAR-10 database as both the traning and testing datasets.

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

transform_crop = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

transform_flip = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),     
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

transform_blur = transforms.Compose([
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 6. Experiment Configurations
experiments = {
    "Baseline": {"dataset": trainset, "transform": transform},
    "Random Label Shuffle": {"dataset": CustomCIFAR10(trainset, noise_type="random_shuffle"), "transform": transform},
    "Label Noise 20%": {"dataset": CustomCIFAR10(trainset, noise_type="label_noise", noise_rate=0.2), "transform": transform},
    "Label Noise 40%": {"dataset": CustomCIFAR10(trainset, noise_type="label_noise", noise_rate=0.4), "transform": transform},
    "Label Noise 60%": {"dataset": CustomCIFAR10(trainset, noise_type="label_noise", noise_rate=0.6), "transform": transform},
    "Label Noise 80%": {"dataset": CustomCIFAR10(trainset, noise_type="label_noise", noise_rate=0.8), "transform": transform},
    "Cropped Images": {"dataset": datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_crop)},
    "Flipped Images": {"dataset": datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_flip)},
    "Blurred Images": {"dataset": datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_blur)}
}

Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 

2. RUNNING TESTS

    We will then use our CNN model defined in model.py where we train the model, test its accuracy then reiterate for 10 epochs.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = {}
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

for exp_name, config in experiments.items():
    print(f"\nRunning experiment: {exp_name}")
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_loader = DataLoader(config["dataset"], batch_size=100, shuffle=True, num_workers=2)
    train(model, device, train_loader, optimizer, criterion, num_epochs=5)
    
    accuracy, class_accuracies, y_true, y_pred = test(model, device, test_loader)
    results[exp_name] = {"accuracy": accuracy, "class_accuracies": class_accuracies}
    print(f"Test Accuracy: {accuracy:.2f}%")

    plot_confusion_matrix(y_true, y_pred, class_names, exp_name)


Running experiment: Baseline
Test Accuracy: 72.72%

Running experiment: Random Label Shuffle
Test Accuracy: 11.54%

Running experiment: Label Noise
Test Accuracy: 66.79%

Running experiment: Cropped Images
Test Accuracy: 71.95%

Running experiment: Flipped Images
Test Accuracy: 71.86%

Running experiment: Blurred Images
Test Accuracy: 71.16%


3. VISUALIZATION

In [ ]:
# Class-wise Accuracy Plot
plt.figure(figsize=(14, 6))
x = np.arange(len(class_names)) 
width = 0.13
excluded = {"Label Noise 40%", "Label Noise 60%", "Label Noise 80%"}
included_experiments = [exp_name for exp_name in results if exp_name not in excluded]
for i, exp_name in enumerate(included_experiments):
    result = results[exp_name]
    plt.bar(x + i*width, result["class_accuracies"], width, label=exp_name)

plt.xlabel('Classes')
plt.ylabel('Accuracy (%)')
plt.title('Class-wise Accuracy for Selected Experiments')
plt.xticks(x + (width * (len(included_experiments) - 1) / 2), class_names, rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig('class_accuracies_filtered.png')
plt.close()

# Overall Accuracy Plot
plt.figure(figsize=(8, 5))
excluded = {"Label Noise 40%", "Label Noise 60%", "Label Noise 80%"}
exp_names = [exp_name for exp_name in results if exp_name not in excluded]
accuracies = [results[exp]["accuracy"] for exp in exp_names]
plt.bar(exp_names, accuracies)
plt.xlabel('Experiment')
plt.ylabel('Overall Accuracy (%)')
plt.title('Overall Accuracy for Different Experiments')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('overall_accuracies.png')
plt.close()

# Label noise level accuracy plot
noise_labels = [
    "Baseline",
    "Label Noise 20%",
    "Label Noise 40%",
    "Label Noise 60%",
    "Label Noise 80%",
    "Random Label Shuffle"
]
x = [0, 20, 40, 60, 80, 100]
y = [results[label]["accuracy"] for label in noise_labels]
plt.figure(figsize=(7, 5))
plt.plot(x, y, marker='o', linestyle='-', color='tab:blue')
plt.xticks(x)
plt.xlabel("Label Noise (%)")
plt.ylabel("Accuracy (%)")
plt.title("Accuracy vs. Label Noise Level")
plt.grid(True)
plt.tight_layout()
plt.savefig("accuracy_vs_label_noise.png")
plt.close()

# 9. Save Results to Text File
with open('experiment_results.txt', 'w') as f:
    for exp_name, result in results.items():
        f.write(f"\nExperiment: {exp_name}\n")
        f.write(f"Overall Accuracy: {result['accuracy']:.2f}%\n")
        f.write("Class-wise Accuracies:\n")
        for cls, acc in zip(class_names, result['class_accuracies']):
            f.write(f"  {cls}: {acc:.2f}%\n")